In [1]:
import pandas as pd
import numpy as np

import folium # map rendering

from geopy.geocoders import Nominatim
from geopy import distance

# import requests

# import math

In [2]:
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.geocode("Rumbach Sebestyén u., Budapest, 1075 Hungary")

print(location.address)
print((location.latitude, location.longitude))
print(location.raw)

Rumbach Sebestyén utca, Erzsébetváros, 7. kerület, Budapest, Közép-Magyarország, 1075, Magyarország
(47.4970308, 19.0598899)
{'place_id': '147830376', 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': '312567851', 'boundingbox': ['47.496647', '47.4970308', '19.0598899', '19.0606059'], 'lat': '47.4970308', 'lon': '19.0598899', 'display_name': 'Rumbach Sebestyén utca, Erzsébetváros, 7. kerület, Budapest, Közép-Magyarország, 1075, Magyarország', 'class': 'highway', 'type': 'residential', 'importance': 0.6199999999999999}


In [3]:
radius = 5
tick = 0.5 

In [4]:
def grid(lat, long, radius, tick):
    """Returns a DataFrame whose rows are a grid of points within a circle around a location
    :param lat: latitude of centre of grid
    :param long: longitude of centre of grid
    :param radius: radius of grid in km
    :param tick: distance between points in grid in km
    :returns: pandas DataFrame with a row for each point in the grid and 
              columns:
                  'Name': string of latitude and longitude of point separated by a comma
                  'Latitude': latitude of point
                  'Longitude': longitude of point
    """
    # get length of 1 degree at current latidude and logitude
    latDegree = distance.distance((lat - 0.5, long), (lat + 0.5, long)).km
    longDegree = distance.distance((lat, long - 0.5), (lat, long + 0.5)).km
    # get length of 1 tick in degrees
    latTick = tick / latDegree
    longTick = tick / longDegree
    # get coordinates of point which is one radius to the South and West of (lat, long)
    bottomLeft = (lat - radius / latDegree, long - radius / longDegree)
    # get tick points along axis
    gridLats = np.arange(2 * radius / tick + 1) * latTick + bottomLeft[0]
    gridLongs = np.arange(2 * radius / tick + 1) * longTick + bottomLeft[1]
    grid = []
    for lat in gridLats:
        for long in gridLongs:
            if distance.distance((location.latitude, location.longitude), (lat, long)).km <= radius:
                grid.append([str(lat) + ', ' + str(long), lat, long])
    grid = pd.DataFrame(grid)
    grid.columns = ['Name', 'Latitude', 'Longitude']
    return grid

In [5]:
help(grid)

Help on function grid in module __main__:

grid(lat, long, radius, tick)
    Returns a DataFrame whose rows are a grid of points within a circle around a location
    :param lat: latitude of centre of grid
    :param long: longitude of centre of grid
    :param radius: radius of grid in km
    :param tick: distance between points in grid in km
    :returns: pandas DataFrame with a row for each point in the grid and 
              columns:
                  'Name': string of latitude and longitude of point separated by a comma
                  'Latitude': latitude of point
                  'Longitude': longitude of point



In [6]:
grid = grid(location.latitude, location.longitude, radius, tick)

In [7]:
# create map
map_grid = folium.Map(location=[location.latitude, location.longitude], zoom_start=12)


for i in grid.index:
    label = folium.Popup(grid.at[i, 'Name'], parse_html=True)
    folium.CircleMarker([grid.at[i, 'Latitude'], grid.at[i, 'Longitude']],
                        radius=1,
                        popup=label,
                        fill=True,
                        fill_opacity=0.5).add_to(map_grid)
       
map_grid

In [12]:
geolocator.reverse((47.45655, 19.05988), exactly_one=False)

[Location(Nagy-Duna sor, Csepel-Szigetcsúcs, 21. kerület, Budapest, Közép-Magyarország, 1211, Magyarország, (47.4550399437689, 19.0616836009134, 0.0))]

In [11]:
from geopy.geocoders import GeocodeEarth
GeocodeEarth.reverse((47.4970308, 19.04661802), exactly_one=False)

TypeError: reverse() missing 1 required positional argument: 'query'